# Scheduling with clashes

Model by [**Vince Knight**](http://vknight.org/).

A university runs 14 modules over three subjects: Art, Biology and Chemistry. Each
subject runs core modules and optional modules. 

The university is required to schedule examinations for each of these modules.
The university would like the exams to be scheduled using the least amount of time
slots. However not all modules can be scheduled at the same time as they share some
students.

Variables: $X_{mt}$ if module $m$ is assigned to day $t$ and the auxiliary variable $Y_t$ if the time slot $t$ is used (we want to minimize to days of the exams). Then we can write the model as:

\begin{align}
\text{Minimise: } \sum_{t \in T} Y_j & \label{eqn:objective_modules} \\
\text{Subject to: } & \nonumber \\
\frac{1}{|M|} \sum_{m \in M} X_{mt} &\leq Y_j \text{ for all } j \in T \label{eqn:auxiliary} \\
\sum_{t \in T} X_{mt} &= 1 \text{ for all } m \in M \label{eqn:schedule_all_modules} \\
\sum_{m \in A_c \cup A_o} X_{mt} &\leq 1 \text{ for all } t \in T \label{eqn:clique1} \\
\sum_{m \in B_c \cup B_o \cup A_c} X_{mt} &\leq 1 \text{ for all } t \in T \label{eqn:clique2} \\
\sum_{m \in B_c \cup B_o \cup C_o} X_{mt} &\leq 1 \text{ for all } t \in T \label{eqn:clique3} \\
\sum_{m \in B_o \cup C_c \cup C_o} X_{mt} &\leq 1 \text{ for all } t \in T \label{eqn:clique4}
\end{align}

In [1]:
library(ROI)
library(data.table)

ROI: R Optimization Infrastructure
Registered solver plugins: nlminb, lpsolve.
Default solver: auto.


In [2]:
#Input
#Modules and time range
mVar = 14
tVar = 14
# Modules and their Clashes
Ac <- c(0, 1)
Ao <- c(2, 3, 4)
Bc <- c(5, 6)
Bo <- c(7, 8)
Cc <- c(9, 10)
Co <- c(11, 12, 13)
list_clashes <- list(
  c(Ac, Ao),
  c(Bc, Bo, Co),
  c(Bc, Bo, Ac),
  c(Bo, Cc, Co)
)

#TODO: Add demand restrinctions

In [3]:
#Build variables and objective
write_objective = function(mVar, tVar){
  varX_col_idx = 1:(mVar*tVar)
  varY_col_idx = ((mVar*tVar)+1):(tVar)
  obj = c(rep(0, mVar*tVar), rep(1, tVar))
  return(list(varX_col_idx,
              varY_col_idx,
              obj))
}

In [4]:
#A function to return the flatened index of the variable
flat_idx = function(m,t,mVar, tVar){
    #m fast
    #t slow
    return(mVar*(t-1) + m)
}

In [5]:
#Build Contraints
#The clash contraints (the last 4)

write_X_clashes = function(m_idx, t_idx, mVar, tVar){
    
    row_idx = rep(0,mVar*tVar + tVar) #the coefficient of all the variables
    clashes = flat_idx(m_idx,t_idx, mVar, tVar)
    row_idx[clashes] = 1
    return(row_idx)
    
}



#Each module is scheduled in one day only (3rd constraint)
write_X_requirements = function(m_idx, mVar, tVar){
   
    row_idx = rep(0,mVar*tVar + tVar) #the coefficient of all the variables
    row_idx[flat_idx(m_idx, 1:tVar, mVar, tVar)] = 1
    return(row_idx)
}

write_Y_requirements = function(t_idx, tVar, mVar){
   
    row_idx = rep(0,mVar*tVar + tVar) #the coefficient of all the variables
    row_idx[flat_idx(1:mVar, t_idx, mVar, tVar)] = 1 #coefficients of X
    row_idx[mVar*tVar + t_idx] = -mVar #coefficients of Y
    return(row_idx)
}


In [6]:
write_constraints = function(list_clashes, mVar, tVar){
  
  all_rows <- c()
  all_dirs <- c()
  all_rhss <- c()
  n_rows <- 0
  
  #clashes contraints
  print("Writing Clash constraints ...")
  for (clash in list_clashes){
    for (t in 1:tVar){
      clashes <- write_X_clashes(clash, t, mVar, tVar)
      all_rows <- append(all_rows, clashes)
      all_dirs <- append(all_dirs, "<=")
      all_rhss <- append(all_rhss, 1)
      n_rows <- n_rows + 1
    }
  }
  print("Writing X requirements ...")
  #Write X requirements
  for (m in 1:mVar){
    reqs <- write_X_requirements(m, mVar, tVar)
    all_rows <- append(all_rows, reqs)
    all_dirs <- append(all_dirs, "==")
    all_rhss <- append(all_rhss, 1)
    n_rows <- n_rows + 1
  }
  
  print("Writing Y requirements ...")
  #Write Y requirements
  for (t_idx in 1:tVar){
    yConstraints <- write_Y_requirements(t_idx, mVar, tVar)
    all_rows <- append(all_rows, yConstraints)
    all_dirs <- append(all_dirs, "<=")
    all_rhss <- append(all_rhss, 0)
    n_rows <- n_rows + 1
  }
  
  f.con <- matrix(all_rows, nrow = n_rows, byrow = TRUE)
  f.dir <- all_dirs
  f.rhs <- all_rhss
  return(list(f.con, f.dir, f.rhs))
  
}


In [7]:
constraints <- write_constraints(
  list_clashes = list_clashes,
  mVar = mVar,
  tVar = tVar
)
f.con <- constraints[[1]]
f.dir <- constraints[[2]]
f.rhs <- constraints[[3]]
f.obj <- write_objective(mVar, tVar)[[3]]

[1] "Writing Clash constraints ..."
[1] "Writing X requirements ..."
[1] "Writing Y requirements ..."


In [8]:
#Solve
milp <- OP(
  objective = L_objective(f.obj),
  constraints = L_constraint(L = f.con, dir = f.dir, rhs = f.rhs),
  types = rep("B", length(f.obj)), #All binary
  maximum = FALSE
)
sol <- ROI_solve(milp)
print(sol$solution)

  [1] 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0
 [38] 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 [75] 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
[112] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[149] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
[186] 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 0


In [9]:
varX_col_idx = write_objective(mVar, tVar)[[1]]
varX = sol$solution[varX_col_idx]

get_schedule = function(sol, mVar, tVar){
    
    schedule = data.table(t(array(varX, dim=c(tVar, mVar))))
    colnames(schedule) = paste0('module_', 1:mVar)
    rownames(schedule) = paste0('day_', 1:tVar)
    
    return(schedule)
}


#A pretty function
get_schedule_2 <- function(sol, n_days, n_modules){
  schedule = ""
  for (day in 1:n_days){
    if (sol$solution[(n_days * n_modules) + day] == 1){
      schedule <- paste(schedule, "\n", "Day", day, ":")
      for (module in 1:n_modules){
        var <- ((day - 1) * n_modules) + module
        if (sol$solution[var] == 1){
          schedule <- paste(schedule, module)
        }
      }
    }
  }
  schedule
}


## The optimal schedule (multiple solutions might exist)

In [10]:
get_schedule(sol, mVar, tVar)

,module_1,module_2,module_3,module_4,module_5,module_6,module_7,module_8,module_9,module_10,module_11,module_12,module_13,module_14
day_1,0,1,0,0,0,1,0,0,1,0,0,0,0,0
day_2,0,0,0,0,1,0,0,0,0,1,0,0,0,0
day_3,0,0,1,0,0,0,1,0,0,0,0,0,0,0
day_4,1,0,0,0,0,0,0,0,0,0,0,1,0,0
day_5,0,0,0,0,0,0,0,1,0,0,0,0,0,0
day_6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
day_7,0,0,0,1,0,0,0,0,0,0,0,0,1,1
day_8,0,0,0,0,0,0,0,0,0,0,0,0,0,0
day_9,0,0,0,0,0,0,0,0,0,0,0,0,0,0
day_10,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
cat(get_schedule_2(sol = sol, n_days = tVar, n_modules = mVar))

 
 Day 1 : 2 6 9 
 Day 2 : 5 10 
 Day 3 : 3 7 
 Day 4 : 1 12 
 Day 5 : 8 
 Day 7 : 4 13 14 
 Day 13 : 11